In [ ]:
# Basic
import torch
import math
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import numpy as np
from numpy import random
torch.set_num_threads(4)

# PDEi
from utils_pde.utils_pde_2dpoisson import Poisson2D

# Viz
from utils_tools.utils_result_viz import plot_predictions_2D

# Base Mdoels
from utils_uqmd.utils_uq_dropout import DropoutPINN
from utils_uqmd.utils_uq_mlp import MLPPINN
from utils_uqmd.utils_uq_vi import VIBPINN

# CP
from utils_uqmd.utils_uq_cp import CP

# Ensure reproducibility
import random, numpy as np, torch
seed = 12345
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# ----------------------------------------------------------------------

# Data Noise
data_noise = 0.05

# Define PDE
domain=((0.0, 1.0), (0.0, 1.0))
true_solution=lambda xy: torch.sin(math.pi * xy[..., 0:1]) * torch.sin(math.pi * xy[..., 1:2])
pde = Poisson2D(domain, true_solution)

# Generate training and testing data of the poisson function
(X_train, Y_train) = pde.data_generation(500, data_noise)
(X_test, Y_test) = pde.data_generation(100, data_noise)
(X_calibration, Y_calibration) = pde.data_generation(200, data_noise)

# Number of colocation point
colloc_pt_num = 100

# ----------------------------------------------------------------------

# Build base model
do_pinn = DropoutPINN(
    pde_class=pde, 
    input_dim=2,
    hidden_dims=[32, 64, 128, 128, 128, 64, 32], 
    output_dim=1, 
    p_drop=0.1
)

# Define tuning arguments
# Base Model
fit_args = {
    "coloc_pt_num":colloc_pt_num,
    "X_train":X_train, 
    "Y_train":Y_train
}

# Change the parameter grids to test
fit_kwargs_grid = {
    "epochs":[200],
    "λ_pde":[3.0, 5.0], "λ_bc":[5.0, 10.0], "λ_data":[1.0, 2.0, 5.0],  # Change based on baseline model
    "lr":[1e-3, 3e-4],
    # "scheduler_cls":[StepLR], "scheduler_kwargs":[{'step_size': 5000, 'gamma': 0.5}],
    "stop_schedule":[10000, 20000, 40000]
}

baseline_pred_kwargs = {  # Change all the kwargs here
    "n_samples":100 
}

# CP Model
cp_pred_kwargs = {
        "X_train":X_train,  "Y_train":Y_train,
        "X_cal":X_calibration, "Y_cal":Y_calibration,
        "heuristic_u":"raw_std",
        "k":10
}

cp_testing_args = {
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test, 
    "X_cal":X_calibration, "Y_cal":Y_calibration, "X_train":X_train, "Y_train":Y_train, 
    "heuristic_u":"raw_std", # Change base on if the baseline model has its original uq band
    "k":1
}

baseline_testing_args = { 
    "uqmodel":do_pinn,   # Change this
    "alphas":alphas, 
    "X_test":X_test, "Y_test":Y_test,
    "n_samples":100
}

# ----------------------------------------------------------------------

# Predicting grid
n_grid = 100
x = torch.linspace(0, 1, n_grid)
y = torch.linspace(0, 1, n_grid)
X, Y = torch.meshgrid(x, y, indexing='xy')
grid_test = torch.cat([X.reshape(-1, 1), Y.reshape(-1, 1)], dim=1)

from utils_tools.utils_result_metrics import generating_alphas
alphas = generating_alphas(10)

# ----------------------------------------------------------------------

# Start hyperparameter tuning 
from utils_tools.utils_tuning import hyperparameter_tuning
from utils_tools.utils_result_viz import plot_2D_comparison_with_coverage
from utils_uqmd.utils_uq_cp import CP
from utils_tools.utils_result_metrics import cp_test_uncertainties, vi_test_uncertainties, do_test_uncertainties

hyperparameter_tuning(
    plot_title="Dopout CP Model", # Change this
    # Model Fitting & Predicting
    uqmodel=do_pinn,  # Change this
    alpha=0.05, 
    X_test=grid_test, Y_test=Y_test, 
    fit_args=fit_args, fit_kwargs_grid=fit_kwargs_grid, baseline_pred_kwargs=baseline_pred_kwargs, cp_pred_kwargs=cp_pred_kwargs, 
    true_solution=pde.true_solution,
    # Coverage Test
    baseline_testing_args=baseline_testing_args, cp_testing_args=cp_testing_args,
    baseline_test_uncertainties=do_test_uncertainties, # Change this
    # Plotting function
    plotting_func=plot_2D_comparison_with_coverage, # 
    save_dir="2dpoisson_do_cp",  # Change this
    X_validation=X_test, Y_validation=Y_test
)


[🔎] Trying: {'epochs': 200, 'λ_pde': 3.0, 'λ_bc': 5.0, 'λ_data': 1.0, 'lr': 0.001, 'stop_schedule': 10000}

[🟠] Training...
ep     1 | L=3.28e+02 | data=1.71e-01 | pde=1.09e+02  ic=0.00e+00  bc=1.42e-02 | lr=1.00e-03

[🟠] Inferencing...


/Users/yifanyu/miniconda3/envs/cp/lib/python3.10/site-packages/torch/utils/_device.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return func(*args, **kwargs)



[🟠] Computing Coverage...


100%|██████████| 8/8 [00:01<00:00,  7.33it/s]



[✅] Data Loss = 1.881e-01

[🔎] Trying: {'epochs': 200, 'λ_pde': 3.0, 'λ_bc': 5.0, 'λ_data': 1.0, 'lr': 0.001, 'stop_schedule': 20000}

[🟠] Training...
ep     1 | L=2.72e+01 | data=1.90e-01 | pde=8.66e+00  ic=0.00e+00  bc=2.03e-01 | lr=1.00e-03


KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>